In [ ]:
#ライブラリのインポート
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## GPUによる高速化

TensorflowやPyTorchといった深層学習ライブラリはGPUでの実行がサポートされています。
GPUを使うことで大規模計算が高速化できることがあります。

このノートブックではTensorflow/KerasやPyTorchをGPU上で実行し、CPUでの実行したときと計算速度を比較します。


### GPUの利用状況の確認

現在接続しているノードのGPU使用率は`nvidia-smi`コマンドで確認できます。
```bash
$ nvidia-smi
Fri Jul 28 17:22:22 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB           Off| 00000000:13:00.0 Off |                   On |
| N/A   24C    P0               51W / 400W|    877MiB / 40960MiB |     N/A      Default |
|                                         |                      |              Enabled |
+-----------------------------------------+----------------------+----------------------+

+---------------------------------------------------------------------------------------+
| MIG devices:                                                                          |
+------------------+--------------------------------+-----------+-----------------------+
| GPU  GI  CI  MIG |                   Memory-Usage |        Vol|      Shared           |
|      ID  ID  Dev |                     BAR1-Usage | SM     Unc| CE ENC DEC OFA JPG|
|                  |                                |        ECC|                       |
|==================+================================+===========+=======================|
|  0    7   0   0  |              12MiB /  4864MiB  | 14      0 |  1   0    0    0    0 |
|                  |               0MiB /  8191MiB  |           |                       |
+------------------+--------------------------------+-----------+-----------------------+
|  0    8   0   1  |              12MiB /  4864MiB  | 14      0 |  1   0    0    0    0 |
|                  |               0MiB /  8191MiB  |           |                       |
+------------------+--------------------------------+-----------+-----------------------+
|  0    9   0   2  |             802MiB /  4864MiB  | 14      0 |  1   0    0    0    0 |
|                  |               2MiB /  8191MiB  |           |                       |
+------------------+--------------------------------+-----------+-----------------------+
|  0   10   0   3  |              12MiB /  4864MiB  | 14      0 |  1   0    0    0    0 |
|                  |               0MiB /  8191MiB  |           |                       |
+------------------+--------------------------------+-----------+-----------------------+
|  0   11   0   4  |              12MiB /  4864MiB  | 14      0 |  1   0    0    0    0 |
|                  |               0MiB /  8191MiB  |           |                       |
+------------------+--------------------------------+-----------+-----------------------+
|  0   12   0   5  |              12MiB /  4864MiB  | 14      0 |  1   0    0    0    0 |
|                  |               0MiB /  8191MiB  |           |                       |
+------------------+--------------------------------+-----------+-----------------------+
|  0   13   0   6  |              12MiB /  4864MiB  | 14      0 |  1   0    0    0    0 |
|                  |               0MiB /  8191MiB  |           |                       |
+------------------+--------------------------------+-----------+-----------------------+
                                                                                         
+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |
|        ID   ID                                                             Usage      |
|=======================================================================================|
|    0    9    0      11117      C   .../local/venv/deeplearning/bin/python      782MiB |
+---------------------------------------------------------------------------------------+
```
このノードではGPU1台が搭載されていますが、複数のユーザーで共有して使用できるようにGPUを7分割しています。
１つ目のブロックから、NVIDIA A100というGPUが載っていて、メモリは40960MiB ということが読み取れます。
2つ目のブロックでは7分割された各GPUの状態が表示されています。上から3つ目のデバイス(GIID=9, MIG Dev=2)ではGPUメモリが802MiB使用されています。
最後のブロックは、GPUを使用しているプロセスが表示されています。GIID=9 のGPUを1つのプロセスが使用していることが読み取れます。

自分のプロセスがゾンビ状態となってGPUのメモリを専有してしまうことがないように定期的にチェックしてください。

GPUを使っているプロセスを起動したユーザーを確認するコマンドの一例は以下です。
```bash
$ ps -up `nvidia-smi --query-compute-apps=pid --format=csv,noheader | sort -u`
USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
```
ここで表示されるメモリの値はGPU上のメモリではないことに注意してください。

notebook上では"!"をつけることでbashコマンドが実行できます。     

In [ ]:
!nvidia-smi

### 使用するGPUの指定
まずは使うGPUを指定します。このステップなしでもGPUは利用できますが、GPUを専有してしまうことで共有マシンを使用している他のユーザーに迷惑をかけてしまうことがあります。
今回の講習では、使うGPUは1つのみに限定させます。

GPUを指定するには、TensorflowやPyTorchをimportする前に環境変数`CUDA_VISIBLE_DEVICES`にGPUのUUIDをしてしてください。
```python
import os
os.environ['CUDA_VISIBLE_DEVICES'] = 'MIG-554c4086-6ae7-5b25-8c39-e5980d23ae92'
```
UUIDは以下のようにして確認することができます。
```bash
[saito@centos7 ~]$ nvidia-smi -L
GPU 0: NVIDIA A100-SXM4-40GB (UUID: GPU-07ce7acc-c626-f86c-3b8c-170a84e404b3)
  MIG 1g.5gb      Device  0: (UUID: MIG-554c4086-6ae7-5b25-8c39-e5980d23ae92)
  MIG 1g.5gb      Device  1: (UUID: MIG-9b9d3d09-b7d8-5351-af1e-5607f0fbd02a)
  MIG 1g.5gb      Device  2: (UUID: MIG-0dfbf33f-6966-5b43-a167-2cb7be339805)
  MIG 1g.5gb      Device  3: (UUID: MIG-1baae807-2fcd-5438-8a89-d320680ade08)
  MIG 1g.5gb      Device  4: (UUID: MIG-cedbde95-8c79-51e0-b848-7bf73342a233)
  MIG 1g.5gb      Device  5: (UUID: MIG-84f9461f-49b6-5343-a3b9-5be94986cdad)
  MIG 1g.5gb      Device  6: (UUID: MIG-95045bde-aa51-5831-8983-63dbf0262e77)
```
Device ID(上の例だと0 ~ 6)と`nvidia-smi`の出力を見比べて、使用されて**いない**GPUを選択するようにしてください。

In [ ]:
# 使うGPUを一つに制限します。
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = 'MIG-554c4086-6ae7-5b25-8c39-e5980d23ae92'

In [ ]:
# PyTorchが使うCPUの数を制限します。(VMを使う場合)
import os
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'

from torch import set_num_threads, set_num_interop_threads
num_threads = 1
set_num_threads(num_threads)
set_num_interop_threads(num_threads)

### GPU上でのPyTorchの計算
PyTorchでは入力データ(tensor)やモデルをどのデバイスで計算させるか明示的に指定する必要があります。
tensorやモデルオブジェクトに`.cuda()`、または`.to('cuda')`を指定することでGPUメモリにオブジェクトをコピーできます。

#### tensor オブジェクトのGPUメモリへの移動
```python
from torch import tensor
tensor(1)  # CPU
tensor(1).cpu()  # CPU
tensor(1).cuda()  # GPU
tensor(1).to('cpu')  # CPU
tensor(1).to('cuda')  # GPU
```

#### Model オブジェクトのGPUメモリへの移動
```python
from torch.nn import Sequential
from torch.nn import Linear
from torch.nn import Sigmoid
from torch.nn import ReLU
model = Sequential(
    Linear(in_features=2, out_features=32),
    Sigmoid(),
)

model  # CPU
model.cpu()  # CPU
model.cuda()  # GPU
model.cpu()  # CPU
model.to('cpu')  # CPU
model.to('cuda')  # GPU
```

## MLPのトレーニング

MLPのトレーニングをCPU/GPUでそれぞれ実行することで、計算時間の変化を確認します。

データセットは乱数で適当に作成します。

In [ ]:
from numpy.random import default_rng
rng = default_rng(seed=0)

# 100万行、入力次元100のランダムなデータを作成。
datasize = 1000000
x = rng.normal(size=(datasize, 100))
t = rng.integers(0, 2, size=(datasize, 1))

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
from torch import tensor
ds = TensorDataset(tensor(x).float(), tensor(t).float())
dataloader = DataLoader(ds, batch_size=1024, shuffle=True)

## CPUによるサンプルコード

PyTorchでは明示的に指定しない限り、CPU上で計算が行われます。

In [ ]:
from torch.nn import Sequential
from torch.nn import Linear
from torch.nn import Sigmoid
from torch.nn import ReLU
from torch.nn import BCELoss
from torch.optim import Adam
from torch import from_numpy
from torch.cuda import synchronize
import time

# モデルの定義
model = Sequential(
    Linear(in_features=100, out_features=256),  # ノード数が256の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=256, out_features=256),  # ノード数が256の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=256, out_features=256),  # ノード数が256の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=256, out_features=256),  # ノード数が256の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=256, out_features=256),  # ノード数が256の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=256, out_features=1),  # ノード数が1の層を追加。
    Sigmoid(),  # 活性化関数はシグモイド関数。
)
# 誤差関数としてクロスエントロピーを指定。最適化手法は(確率的)勾配降下法
loss_fn = BCELoss()
optimizer = Adam(model.parameters())

# トレーニング
for i_epoch in range(5):

    # エポックごとのロス、accuracyを計算するための変数
    loss_total = 0.

    # 時間の測定
    synchronize()
    start_time = time.time()

    for x, t in dataloader:
        # 順伝搬
        y_pred = model(x)

        # ロスの計算
        loss = loss_fn(y_pred, t)
        loss_total += loss.detach().numpy()

        # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
        # これをしないと、勾配の値はそれまでの値との和がとられる。
        optimizer.zero_grad()

        # 誤差逆伝播。各パラメータの勾配が計算される。
        loss.backward()

        # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
        optimizer.step()

    # ロス、accuracyをミニバッチの数で割って平均を取ります。
    loss_total /= len(dataloader)
    synchronize()  # GPUの処理が終わるのを待ちます。
    print(f'epoch = {i_epoch}, time = {time.time() - start_time: .3f} sec, loss = {loss_total}')



## GPUによるサンプルコード
tensorやmodelをGPU上に移動する(`.cuda()`の追加)ことで、GPUを使ったトレーニングができます。

In [ ]:
from torch.nn import Sequential
from torch.nn import Linear
from torch.nn import Sigmoid
from torch.nn import ReLU
from torch.nn import BCELoss
from torch.optim import Adam
from torch import from_numpy
from torch.cuda import synchronize
import time

# モデルの定義
model = Sequential(
    Linear(in_features=100, out_features=256),  # ノード数が256の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=256, out_features=256),  # ノード数が256の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=256, out_features=256),  # ノード数が256の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=256, out_features=256),  # ノード数が256の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=256, out_features=256),  # ノード数が256の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=256, out_features=1),  # ノード数が1の層を追加。
    Sigmoid(),  # 活性化関数はシグモイド関数。
)
# 誤差関数としてクロスエントロピーを指定。最適化手法は(確率的)勾配降下法
loss_fn = BCELoss()
optimizer = Adam(model.parameters())

# GPUメモリにモデルをコピーします。
model = model.cuda()

# トレーニング
for i_epoch in range(5):

    # エポックごとのロス、accuracyを計算するための変数
    loss_total = 0.

    # 時間の測定
    synchronize()
    start_time = time.time()

    for x, t in dataloader:
        # GPUメモリにデータをコピーします。
        x = x.cuda()
        t = t.cuda()

        # 順伝搬
        y_pred = model(x)

        # ロスの計算
        loss = loss_fn(y_pred, t)
        loss_total += loss.detach().cpu().numpy()  # lossの合計値を記録するため、値をCPUにコピーします。

        # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
        # これをしないと、勾配の値はそれまでの値との和がとられる。
        optimizer.zero_grad()

        # 誤差逆伝播。各パラメータの勾配が計算される。
        loss.backward()

        # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
        optimizer.step()

    # ロス、accuracyをミニバッチの数で割って平均を取ります。
    loss_total /= len(dataloader)
    synchronize()  # GPUの処理が終わるのを待ちます。
    print(f'epoch = {i_epoch}, time = {time.time() - start_time: .3f} sec, loss = {loss_total}')



計算時間はどのように変化したでしょうか。
モデルサイズが変化すると、計算時間はどのように変わるでしょうか？